In [42]:
import pandas as pd
from pathlib import Path
from pandas._libs.tslibs.nattype import NaTType
import re

In [39]:
INPUT= Path("files\input\solicitudes_de_credito.csv")
OUTPUT = Path("files\output\solicitudes_de_credito.csv")
OUTPUT.parent.mkdir(exist_ok=True)  

In [65]:
df1=pd.read_csv(INPUT, sep=";", encoding="latin1", low_memory=False)
df1

,Unnamed: 0,sexo,tipo_de_emprendimiento,idea_negocio,barrio,estrato,comuna_ciudadano,fecha_de_beneficio,monto_del_credito,lÃ­nea_credito
0,0,masculino,comercio,fabrica de,prado,3,10.0,13/07/2018,5000000,microempresarial
1,1,femenino,servicio,recreativo-educativo,barrio caicedo,1,9.0,30/10/2017,6000000,microempresarial
2,2,femenino,comercio,tienda,aranjuez,2,4.0,3/08/2017,7300000,microempresarial
3,3,Masculino,comercio,variedades,cabecera san cristobal,3,60.0,20/11/2017,7000000,microempresarial
4,4,femenino,comercio,fabrica de,robledo,2,7.0,23/02/2017,5000000,microempresarial
...,...,...,...,...,...,...,...,...,...,...
10915,10915,masculino,agropecuaria,comidas rapidas,villatina,1,8.0,9/05/2018,7000000,microempresarial
10916,10916,masculino,comercio,fabrica de arepas,villa hermosa,3,8.0,23/01/2017,7377000,microempresarial
10917,10917,femenino,comercio,variedades,cabecera san cristobal,1,60.0,12/04/2018,7800000,microempresarial
10918,10918,femenino,servicio,cafe e internet,manrique oriental,2,3.0,1/06/2017,7370000,microempresarial


In [ ]:
df= pd.read_csv(INPUT, sep=";")
df.drop(columns=["Unnamed: 0"], inplace=True)
df["linea_credito"]= df["línea_credito"]
df.drop(columns=["línea_credito"], inplace=True)


,sexo,tipo_de_emprendimiento,idea_negocio,barrio,estrato,comuna_ciudadano,fecha_de_beneficio,monto_del_credito,linea_credito
0,masculino,comercio,fabrica de,prado,3,10.0,13/07/2018,5000000,microempresarial
1,femenino,servicio,recreativo-educativo,barrio caicedo,1,9.0,30/10/2017,6000000,microempresarial
2,femenino,comercio,tienda,aranjuez,2,4.0,3/08/2017,7300000,microempresarial
3,Masculino,comercio,variedades,cabecera san cristobal,3,60.0,20/11/2017,7000000,microempresarial
4,femenino,comercio,fabrica de,robledo,2,7.0,23/02/2017,5000000,microempresarial
...,...,...,...,...,...,...,...,...,...
10915,masculino,agropecuaria,comidas rapidas,villatina,1,8.0,9/05/2018,7000000,microempresarial
10916,masculino,comercio,fabrica de arepas,villa hermosa,3,8.0,23/01/2017,7377000,microempresarial
10917,femenino,comercio,variedades,cabecera san cristobal,1,60.0,12/04/2018,7800000,microempresarial
10918,femenino,servicio,cafe e internet,manrique oriental,2,3.0,1/06/2017,7370000,microempresarial


In [58]:
_pat = re.compile(r"[^\d]+")     # separador: cualquier no-dígito

def fecha_mes_en_medio(celda) -> pd.Timestamp | NaTType:
    """
    Interpreta fechas donde el segundo número es SIEMPRE el mes.
    Ejemplos válidos:
        2018/05/3   → 2018-05-03
        3-05-2018   → 2018-05-03
        7.12.21     → 2021-12-07
    Devuelve pd.NaT si algo no encaja.
    """
    if pd.isna(celda):
        return pd.NaT

    partes = _pat.split(str(celda).strip())
    if len(partes) != 3 or not all(p.isdigit() for p in partes):
        return pd.NaT

    a, m, b = map(int, partes)   # m = mes por regla

    # Determinar año (4 dígitos o > 31) y día
    if a > 31 or a >= 1000:
        anio, dia = a, b
    elif b > 31 or b >= 1000:
        anio, dia = b, a
    else:
        # Ambos son de dos dígitos → asumimos formato YY-M-D (>= 70 ⇒ 19xx, si no 20xx)
        anio, dia = (1900 + a if a >= 70 else 2000 + a), b

    # Validaciones básicas
    if not (1 <= m <= 12 and 1 <= dia <= 31):
        return pd.NaT

    try:
        return pd.Timestamp(anio, m, dia)
    except ValueError:           # día fuera de rango para ese mes
        return pd.NaT

# Uso vectorizado
df["fecha_de_beneficio"] = df["fecha_de_beneficio"].apply(fecha_mes_en_medio)


In [59]:
df['monto_del_credito'] = (
    df['monto_del_credito']  # columna original
      .astype(str)                       # asegura string
      .str.replace('[\$,]', '', regex=True) # quita $ y comas de miles
      .str.replace('.', '',  regex=False)   # (si el separador miles es punto)
      .str.replace(',', '.', regex=False)   # y el decimal es coma
      .astype(float)                      # convertir a número flotante
)

# Monto debe ser positivo y < 1e9
mask_monto_ok = df['monto_del_credito'].between(0, 1e9)
df = df[mask_monto_ok]

In [62]:
for col in ["sexo","tipo_de_emprendimiento","idea_negocio","barrio","linea_credito"]:
    df[col] = (
        df[col]
          .str.strip()          # quita espacios en extremos
          .str.lower()          # minúsculas uniformes
    )

In [64]:
df.to_csv(OUTPUT, index=False)